In [ ]:
import json
import pickle
import json
from askem.elastic import get_text
from askem.retriever.base import get_client

In [11]:
# We patched the data for topic: ["criticalmaas", "geoarchive"] in 240803
# This are the results

with open("tmp/patch_status.json") as f:
    patch_status = json.load(f)

for k, v in patch_status.items():
    print(f"{k}: {len(v)}")

success: 49622
fail: 34219


In [ ]:
### DANGER: REMOVE FAILED PATCH OBJECTS MANUALLY

# WEAVIATE_CLIENT = get_client()

# # Only 10000 paragarph can be remove at a time. So we need to do it in batch
# for _ in tqdm(range(199)):
#     WEAVIATE_CLIENT.batch.delete_objects(
#         class_name="Paragraph",
#         where={
#             "path": ["paper_id"],
#             "operator": "ContainsAny",
#             "valueTextArray": patch_status['fail']
#         },
#     )

In [14]:
# After I deleted all failed doc_ids and rerun the ingest script to take care of the unsucessful patch (which may due to underlying data changes, or older ingest error)
# However, there are still quite a few doc_ids with empty doc.


## Master empty id list from ingest log
with open("tmp/empty_ids.pkl", "rb") as f:
    empty_ids = set(pickle.load(f))

fail_and_empty = []
other = []

# Compare against failed patch doc_ids
for doc_id in patch_status['fail']:
    if doc_id in empty_ids:
        fail_and_empty.append(doc_id)
    else:
        other.append(doc_id)

print(f"fail_and_empty: {len(fail_and_empty)}")
print(f"other: {len(other)}")

fail_and_empty: 28708
other: 5511


In [16]:
# One empty doc example
t = get_text("661a81fd1a703effc6aaa743")
print(t[:200])